Use this with the `cyto_39` environment (the scivision model needs a specific version of `pytorch` that isn't packaged for >3.9, i have raised a Github issue asking if they plan to update it)

`conda env create -f environment.yml`
`conda activate cyto_39`

In [109]:
import os
from scivision import load_pretrained_model, load_dataset
from dotenv import load_dotenv
import torch
import torchvision
import chromadb
load_dotenv()  # sets our object store endpoint and credentials from the .env file

True

In [67]:
dataset = load_dataset(f"{os.environ.get('ENDPOINT', '')}/metadata/intake.yml")
model = load_pretrained_model("https://github.com/alan-turing-institute/plankton-cefas-scivision")
dataset.test_image().to_dask()

<xarray.DataArray (y: 24, x: 15, channel: 3)> Size: 1kB
dask.array<xarray-<this-array>, shape=(24, 15, 3), dtype=uint8, chunksize=(24, 15, 3), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 192B 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * x        (x) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * channel  (channel) int64 24B 0 1 2

The scivision wrapper depends on this being an xarray Dataset with settable attributes, rather than a DataArray

Setting exif_tags: True (Dataset) or False (DataArray) is what controls this
https://docs.xarray.dev/en/stable/generated/xarray.DataArray.to_dataset.html 

https://github.com/alan-turing-institute/scivision/blob/07fb74e5231bc1d56cf39df38c19ef40e3265e4c/src/scivision/io/reader.py#L183
https://github.com/intake/intake/blob/29c8878aa7bf6e93185e2c9639f8739445dff22b/intake/__init__.py#L101

But now we're dependent on image height and width metadata being set in the EXIF tags to use the `predict` interface, this is set in the model description through `scivision`, this is brittle

https://github.com/alan-turing-institute/plankton-cefas-scivision/blob/main/resnet50_cefas/model.py#L71


A quick look at the example dataset that comes with the model, for reference

In [33]:
import requests
print(requests.get(target_datasource.url.item()).text)

sources:
  plankton:
      description: Load example images of plankton from COPEPEDIA public dataset
      origin: 
      driver: intake_xarray.image.ImageSource
      args:
        urlpath: ["zip://*.tif::https://zenodo.org/record/6143685/files/images.zip"]
        chunks: {}
        storage_options: {'anon': True}
        coerce_shape: [1000, 1000]
        exif_tags: True



In [30]:
from scivision.catalog import default_catalog
model_name = 'resnet50-plankton'
compatible_datasources = default_catalog.compatible_datasources(model_name).to_dataframe()
target_datasource = compatible_datasources.loc[compatible_datasources['name'] == 'cefas-plankton']
cat = load_dataset(target_datasource.url.item()) 
dataset = cat.plankton().to_dask()
dataset


/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/intake_xarray/image.py:474: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(ds2.dims),


<xarray.Dataset> Size: 78MB
Dimensions:                               (concat_dim: 26, y: 1000, x: 1000,
                                           channel: 3)
Coordinates:
  * concat_dim                            (concat_dim) int64 208B 0 1 ... 24 25
  * y                                     (y) int64 8kB 0 1 2 3 ... 997 998 999
  * x                                     (x) int64 8kB 0 1 2 3 ... 997 998 999
  * channel                               (channel) int64 24B 0 1 2
Data variables: (12/23)
    raster                                (concat_dim, y, x, channel) uint8 78MB dask.array<chunksize=(1, 1000, 1000, 3), meta=np.ndarray>
    EXIF Image ImageWidth                 (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF Image ImageLength                (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF Image BitsPerSample              (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF Image Compression                (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF Image PhotometricInterpretation  (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    ...                                    ...
    EXIF GPS GPSVersionID                 (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF GPS GPSLatitudeRef               (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF GPS GPSLatitude                  (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF GPS GPSLongitudeRef              (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF GPS GPSLongitude                 (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>
    EXIF Image GPSInfo                    (concat_dim) object 208B dask.array<chunksize=(1,), meta=np.ndarray>

In this case we don't want to use the `predict` interface anyway (one of N class labels) - we want the features that go into the last fully-connected layer (as described here https://stackoverflow.com/a/52548419)

In [49]:
network = torch.nn.Sequential(*(list(model._plumbing.model.pretrained_model.children())[:-1]))

In [50]:
network


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [92]:
imgs = dataset.test_image().to_dask()
i= imgs.to_numpy()
i.shape


(24, 15, 3)

https://github.com/alan-turing-institute/plankton-cefas-scivision/blob/main/resnet50_cefas/data.py 


Pass the image through our truncated network and get some embeddings out

In [154]:
embeddings = list(feats[0].squeeze(1).squeeze(1).detach().numpy().astype(float))

In [155]:
embeddings

[0.18681475520133972,
 0.0,
 0.39956235885620117,
 0.0,
 0.09887093305587769,
 0.0,
 0.0,
 0.04044981300830841,
 0.41290155053138733,
 0.5317279100418091,
 1.0021417140960693,
 0.31051135063171387,
 0.0022781193256378174,
 0.0,
 0.0,
 0.07445969432592392,
 0.0,
 0.0,
 0.865257203578949,
 0.09340490400791168,
 0.0,
 2.9403414726257324,
 0.0,
 0.0,
 0.279595285654068,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.92582368850708,
 0.0,
 1.144113302230835,
 1.0001882314682007,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0007738349377177656,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.879579782485962,
 0.0,
 0.0,
 0.29063326120376587,
 0.0,
 0.0,
 0.5436230897903442,
 0.6394219994544983,
 1.206394910812378,
 0.0,
 0.0861923098564148,
 0.04212423786520958,
 0.0,
 1.79121732711792,
 0.0,
 0.0,
 0.0,
 1.24888277053833,
 0.0,
 0.0,
 0.0,
 0.42207008600234985,
 0.04022020101547241,
 0.0,
 0.0,
 0.2464810013771057,
 1.0166819095611572,
 1.4085975885391235,
 0.0,
 0.7129244208335876,
 0.5857464671134949,
 0.0,
 0.0,
 0.

In [100]:
o = torch.stack([torchvision.transforms.ToTensor()(i)])
feats = network(o)
feats.shape

torch.Size([1, 2048, 1, 1])

In [156]:
print(set([type(x) for x in embeddings]))

{<class 'numpy.float64'>}


There's been some guesswork up to this point (honestly expected the input would be resized, and concerned it's not normalised) but we have a set of embeddings, at which point pop them in a vector database and see if similarity queries yield any kind of meaningful results, iterate from there

https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/ - langchain has a few, Milvus would be another option
https://python.langchain.com/v0.1/docs/integrations/vectorstores/ - there are so many integrations!
https://www.trychroma.com/ - this looks straightforward to approach

In [111]:
client = chromadb.PersistentClient(path="./vectors")
collection = client.create_collection(
        name="test_collection",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [163]:
collection.add(
    documents=["test_image"],
    embeddings=[embeddings],
    metadatas=[{"useful": "maybe"}],
    ids=["id1"]  # must be unique, are they required?
)

InvalidDimensionException: Embedding dimension 3 does not match collection dimensionality 2048

In [164]:
collection.get('id1',include=["embeddings"])

{'ids': ['id1'],
 'embeddings': [[0.18681475520133972,
   0.0,
   0.39956235885620117,
   0.0,
   0.09887093305587769,
   0.0,
   0.0,
   0.04044981300830841,
   0.41290155053138733,
   0.5317279100418091,
   1.0021417140960693,
   0.31051135063171387,
   0.0022781193256378174,
   0.0,
   0.0,
   0.07445969432592392,
   0.0,
   0.0,
   0.865257203578949,
   0.09340490400791168,
   0.0,
   2.9403414726257324,
   0.0,
   0.0,
   0.279595285654068,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.92582368850708,
   0.0,
   1.144113302230835,
   1.0001882314682007,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0007738349377177656,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.879579782485962,
   0.0,
   0.0,
   0.29063326120376587,
   0.0,
   0.0,
   0.5436230897903442,
   0.6394219994544983,
   1.206394910812378,
   0.0,
   0.0861923098564148,
   0.04212423786520958,
   0.0,
   1.79121732711792,
   0.0,
   0.0,
   0.0,
   1.24888277053833,
   0.0,
   0.0,
   0.0,
   0.4220700860